## CS230 Deep Learning  | Model Baseline 

Fall 2019: Sheun Aluko, Surya Narayanan, Vadim Piccini

###  Import Dependencies 

Note: util.py defines several utilities for loading and processing raw data, as well as helper funcionts such as the IOU accuracy metric 

In [ ]:
import util as u
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation
import matplotlib.pyplot as plt
keras = tf.keras

### Load dataset 

In [ ]:
x_train,y_train,x_val,y_val,x_test,y_test = u.data_load(f=1)

In [ ]:
#set params
batch_size=4
num_epochs=1
model_name="v0_e" + str(num_epochs) + "_b" + str(batch_size)

In [ ]:
## https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5
#create model
model = Sequential()

#conv
model.add(Conv2D(32, kernel_size=11, input_shape=(512, 512, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, kernel_size=5))
model.add(Activation('relu'))

#pool
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

#conv 
model.add(Conv2D(128, kernel_size=5))
model.add(Activation('relu'))
model.add(Conv2D(32, kernel_size=5))
model.add(Activation('relu'))

#pool 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(4, activation = None))


In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999), loss='mean_squared_error',metrics=[u.IoU])

In [ ]:
model.summary()

In [ ]:
h = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=4, epochs=num_epochs)

In [ ]:
train_loss = h.history['loss']
test_loss  = h.history['val_loss']
epoch_count = range(1, len(train_loss) + 1)

plt.plot(epoch_count, train_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0,0.2])
plt.show();


In [ ]:
plt.savefig("models/" + model_name + ".png")

In [ ]:
def get_results(x,y) : 
    pred = model.predict(x)
    return (pred, y, y-pred , np.mean( (y-pred)**2) ) 
    

In [ ]:
y_train_p, y_train, train_diff , train_loss = get_results(x_train,y_train)

In [ ]:
y_test_p, y_test, test_diff , test_loss = get_results(x_test,y_test)

In [ ]:
y_val_p, y_val, val_diff , val_loss = get_results(x_val,y_val)

In [ ]:
(y_val_p - y_val)#val_diff

In [ ]:
# serialize model to JSON
def save_model(model,name) : 
    model_json = model.to_json()
    model_name = "models/" + name
    with open(model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_name + ".h5")
    print("Saved model to disk")

In [ ]:
save_model(model,model_name)